# **Lesion normalization using FSL on Windows**
(for questions: Marie Pittet (marie.pittet93@gmail.com)

FSL is an open source software for brain imaging analysis. It runs on Linux, or Mac. On Windows, you can run it through the Windows Subsystem for Linux. Here is a walk-through of the procedure and troubleshooting of frequent issues. Lesion delineation should already be done (e.g. on ITK snap, a wonderful open source software that allows to draw simulatenously on multiple scanning modalities), and you should have your binary lesion masks.

Lesion delineation for stroke: on ITK snap, you can use the semi-automated delineation where you feed samples of white matter, gray matter, CSF, and lesioned tissue and grow bubbles inside the lesion to build a 3D model.

Lesion delineation for TBI: don't even bother with semi-automated tools for complex lesions. manually draw them on each slice, and interpolate to build the 3D model.

A quick note on multimodal lesion delineation: better to draw on T1, even if you're looking at the FLAIR/SWI window predominantly.  

## 1. WSL2

FSL lesion normalization procedure doesn't work on WSL1 because of a bad handling of long numbers which generates the error message below.

To upgrade to WSL 2, do the following procedure on Windows Power Shell. Right-click on it to run it with admin rights.


*Error message when using wsl1:
/home/user/fsl/lib/python3.11/site-packages/numpy/core/getlimits.py:542: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)*

In [ ]:
# Checking WSL version
wsl -l -v

# Enable WSL2
dism.exe /online /enable-feature /featurename:Microsoft-Windows-Subsystem-Linux /all /norestart
dism.exe /online /enable-feature /featurename:VirtualMachinePlatform /all /norestart

# RESTART YOUR SYSTEM HERE othewrwise the changes will not be effective

# Set WSL2 as your default version
wsl --set-default-version 2


Now go on your Linux distribution (Ubuntu is preinstalled on Windows, but you can use others) to open a bash terminal. If it is the first time that you access it, it will ask for a username and a password (the password does not appear when you type, it's normal).

Make sure that you can access your lesion and anatomical files from linux. To see in which directory you are, you can type (ls), and to access a directory, you can type (cd directory_name)

Now you're set to start the real deal !

## 2. Install FSL and add three scripts
Install FSL (https://fsl.fmrib.ox.ac.uk/fsl/docs/#/)
Install the 3 scripts;  lesion_norm_fsl.sh, fsl_anat_alt.sh, optiBET.sh (https://neuroimaging-core-docs.readthedocs.io/en/latest/pages/fsl_anat_normalization-lesion.html)

For the scripts, you may have to create .sh files with the right name, copy paste the code from the website and save them in a directory that is accessible (in bin). Do not forget to allocate the rights to execute and modify them.

In [ ]:
# creating a new shell file (here the lesion_norm_fsl.sh, for example)
touch lesion_norm_fsl.sh

# allocating execute and modyfing rights
sudo chmod a+x lesion_norm_fsl.sh

## 3. Converting DICOM to Nifti

Have a folder with your lesion masks and an anatomical file for each patient (T1 preferably, can be T2, or even CT). If you have your files in dicom format, here is how to convert them to Nifti using an open source tool (dcm2niix). You can have your files in .nii or nii.gz (compressed folder) for the following steps, it doesn't matter.  

In [ ]:
# First download dcm2niix from the MRICron website. Then run this command:
dcm2niix -o output_directory input_directory

# For example, if you want to convert dicom files called t1w_p1.dcm to t1w_p1.nii.gz:
dcm2niix -0 t1wp1.nii.gz t1w_p1.dcm

## 4. Reorient and resample your files
Files don't always have the same x,y,z orientation or sampling parameters (I know, annoying). You have to make sure that they all have the same orientation.

In [ ]:
# If you want to check the orientation of a file (here t1w_p1.nii.gz)
fslhd t1w_p1.nii.gz

# If you want to reorient a file to standard orientation (and add the _reoriented in the name)
fslreorient2std t1w_p1.nii.gz t1w_p1_reoriented.nii.gz

# You may want to run the command on all the anatomical and lesion files at once
for file in *.nii; do
    output="${file%.nii}_reoriented.nii"
    fslreorient2std "$file" "$output"
done

# Resampling to 1mm voxels using flirt

flirt -in t1w_p1_reoriented.nii.gz \
      -ref $FSLDIR/data/standard/MNI152_T1_1mm_brain.nii.gz \
      -out resampled_image.nii.gz \
      -applyisoxfm 1 \
      -interp nearestneighbour

## 5. Run your normalization

You can now finally perform the normalization using your reoriented anatomical and lesion files.

Always check the normalization. Sometimes there are just to many compatibility issues with FSL scripts. In that case, you can always try to normalize your preprocessed files on SPM (Matlab has a 30-day trial version....just saying.)

In [ ]:
lesion_norm_fsl.sh t1w_p1.nii.gz lesion_p1_reoriented_resampled.nii.gz # (resampled to match t1)